In [14]:
import cv2
import time
import darknet as dn
from tqdm import tqdm
import gc
tracker_type = "medianflow"
tracker = cv2.TrackerMedianFlow_create

In [15]:
def dawhtsal(bbox1, bbox2):
    #2 hairtsagnii dawhtsaliig tootsoolno.
    bbox1_x1 = int(bbox1[0])
    bbox1_y1 = int(bbox1[1])
    bbox1_x2 = int(bbox1[0] + bbox1[2])
    bbox1_y2 = int(bbox1[1] + bbox1[3])
    bbox2_x1 = int(bbox2[0])
    bbox2_y1 = int(bbox2[1])
    bbox2_x2 = int(bbox2[0] + bbox2[2])
    bbox2_y2 = int(bbox2[1] + bbox2[3])
    dawhtsal_width = min(bbox1_x2, bbox2_x2) - max(bbox1_x1, bbox2_x1)
    dawhtsal_height = min(bbox1_y2, bbox2_y2) - max(bbox1_y1, bbox2_y1)
    if dawhtsal_width <= 0 or dawhtsal_height <= 0:
        return 0.0
    dawhtsal_area = dawhtsal_width * dawhtsal_height
    bbox1_area = (bbox1_x2 - bbox1_x1) * (bbox1_y2 - bbox1_y1)
    bbox2_area = (bbox2_x2 - bbox2_x1) * (bbox2_y2 - bbox2_y1)
    if bbox1_area < bbox2_area:   
        dawhtsal_huwi = dawhtsal_area / (bbox1_area) 
    else:
        dawhtsal_huwi = dawhtsal_area / (bbox2_area)
    return dawhtsal_huwi

In [16]:
def ogtlol(line1, line2):
    def get_orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
            return 0
        return 1 if val > 0 else 2
    def is_on_segment(p, q, r):
        if q[0] <= max(p[0], r[0]) and q[0] >= min(p[0], r[0]) and \
            q[1] <= max(p[1], r[1]) and q[1] >= min(p[1], r[1]):
            return True
        return False
    p1 = line1[0]
    q1 = line1[1]
    p2 = line2[0]
    q2 = line2[1]
    o1 = get_orientation(p1, q1, p2)
    o2 = get_orientation(p1, q1, q2)
    o3 = get_orientation(p2, q2, p1)
    o4 = get_orientation(p2, q2, q1)
    if o1 != o2 and o3 != o4:
        return True
    if o1 == 0 and is_on_segment(p1, p2, q1):
        return True
    if o2 == 0 and is_on_segment(p1, q2, q1):
        return True
    if o3 == 0 and is_on_segment(p2, p1, q2):
        return True
    if o4 == 0 and is_on_segment(p2, q1, q2):
        return True
    return False

def shugam_shalgah(bbox, line):
    x, y, w, h = bbox
    bbox_line1 = [(x, y), (x + w, y)]
    bbox_line2 = [(x + w, y), (x + w, y + h)]
    bbox_line3 = [(x, y), (x, y + h)]
    bbox_line4 = [(x, y + h), (x + w, y + h)]
    if ogtlol(bbox_line1, line) or \
            ogtlol(bbox_line2, line) or \
            ogtlol(bbox_line3, line) or \
            ogtlol(bbox_line4, line):
        return True
    return False

In [17]:
def video_unshih(path):
    video = cv2.VideoCapture(path)
    width  = video.get(3)  # float
    height = video.get(4) # float
    if not video.isOpened():
        return None
    ok, first_frame = video.read()
    if not ok:
        return None
    return video,first_frame,width,height

In [18]:
def click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt.append((x,y))

In [19]:
def mashin_detection(frame,bbox):
    results = dn.performDetect("temp.png")
    nbbox = []
    for cat, score, bounds in results:
        temp = cat
        if not temp == "car" and not temp == "truck" and not temp == "bus":
            continue
        elif score<0.5:
            continue
        x, y, w, h = bounds
        bbox1 = (int(x - w / 2),int(y - h / 2),w,h)
        flag=0
        for j in bbox:
            if j is None:
                continue
            if dawhtsal(bbox1, j) >= 0.6:
                flag = 1
                break
        if flag == 0:
            nbbox.append(bbox1)
    return nbbox

In [23]:
temp = video_unshih("4.avi")
if temp:
    video,frame,width,height = temp
else:
    print("Videog unshij chadsangui")

In [24]:
# image = frame
# clone = image.copy()
# image_list = []
# cv2.namedWindow("Startline")
# cv2.setMouseCallback("Startline", click)
# start = []
# cnt=0
# while True:
#     tseg=None
#     while True:
#         cv2.imshow("Startline", image)
#         key = cv2.waitKey(1) & 0xFF
#         if key == ord("r"):
#             if len(image_list)==0:
#                 image = frame
#             else:
#                 image = image_list[-1]
#                 image_list.pop()
#         elif key == ord("c"):
#             flag_end=True
#             break
#         if len(tseg) == 2:
#             color = (0, 255, 0)  
#             thickness = 6
#             image = cv2.line(image, refPt[0] , refPt[1] , color, thickness)
#             break
#     if flag_end:
#         break
#     start.append(tseg)
    

In [25]:
# Line zurah
refPt = []
image = frame
clone = image.copy()
cv2.namedWindow("Startline")
cv2.setMouseCallback("Startline", click)
start = []
cnt=0
while cnt<4:
    clone2 = image.copy()
    refPt = []
    flag = 0
    while True:
        cv2.imshow("Startline", image)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("r"):
            image = clone.copy()
            if flag == 0:
                start.pop()
                flag = 1
                cnt-=1
        elif key == ord("c"):
            break
        if len(refPt) == 2:
            color = (0, 255, 0)  
            thickness = 6
            image = cv2.line(image, refPt[0] , refPt[1] , color, thickness)
            break
    start.append(refPt)
    if cnt == 3:
        refPt = []
        while True:
            cv2.imshow("Startline", image)
            key = cv2.waitKey(1) & 0xFF
            if key == ord("r"):
                image = clone.copy()
                if flag == 0:
                    start.pop()
                    flag = 1
            elif key == ord("c"):
                break
            if len(refPt) == 2:
                color = (0, 255, 0)  
                thickness = 6
                image = cv2.line(image, refPt[0] , refPt[1] , color, thickness)
                break
    clone = clone2.copy()
    cnt+=1
cv2.destroyAllWindows()
cv2.namedWindow("Endline")
cv2.setMouseCallback("Endline", click)
clone = image.copy()
end = []
cnt = 0
while cnt<4:
    clone2 = image.copy()
    refPt = []
    flag = 0
    while True:
        cv2.imshow("Endline", image)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("r"):
            image = clone.copy()
            if flag == 0:
                end.pop()
                flag = 1
        elif key == ord("c"):
            break
        if len(refPt) == 2:
            color = (0, 0, 255)  
            thickness = 6
            image = cv2.line(image, refPt[0] , refPt[1] , color, thickness)
            break
    end.append(refPt)
    if cnt == 3:
        refPt = []
        while True:
            cv2.imshow("Endline", image)
            key = cv2.waitKey(1) & 0xFF
            if key == ord("r"):
                image = clone.copy()
                if flag == 0:
                    end.pop()
                    flag = 1
            elif key == ord("c"):
                break
            if len(refPt) == 2:
                color = (0, 0, 255)  
                thickness = 6
                image = cv2.line(image, refPt[0] , refPt[1] , color, thickness)
                break
    clone = clone2.copy()
    cnt+=1
cv2.destroyAllWindows()

In [26]:
# print(start,end)
# Tsag uur (start ,endline)
# start = [[(1383, 362), (1396, 375)], [(1378, 481), (1043, 471)], [(756, 366), (736, 397)], [(833, 283), (1099, 321)]]
# end = [[(1481, 376), (1483, 408)], [(701, 506), (934, 517)], [(746, 303), (734, 352)], [(1194, 273), (1263, 273)]]
# dulguun nuur (start ,endline)
# start = [[(852, 338), (893, 409)], [(938, 567), (253, 556)], [(78, 384), (3, 428)], [(278, 308), (555, 321)]] 
# end = [[(973, 391), (998, 449)], [(3, 448), (38, 531)], [(45, 330), (0, 368)], [(595, 280), (702, 272)]]
# Sapporo
# start = [[(78, 326), (457, 323)], [(719, 259), (1134, 416)], [(1351, 489), (1056, 660)], [(304, 504), (450, 707)]]
# end = [[(522, 302), (643, 286)], [(1328, 387), (1496, 429)], [(690, 924), (1132, 850)], [(14, 337), (49, 456)]]

In [9]:
# cv2.imshow("e", frame)
# cv2.waitKey(10000)
# cv2.destroyAllWindows()

In [27]:
start_time = time.time()
bbox=[]
mashin_trackers=[]
cv2.imwrite("temp.png", frame)
results = dn.performDetect("temp.png")
for cat, score, bounds in results:
    temp = cat
    if not temp == 'car' and not temp == 'truck' and not temp == 'bus':
        continue
    elif score<0.5:
        continue
    x, y, w, h = bounds
    bbox.append((int(x - w / 2),int(y - h / 2),w,h)) 
    mashin_trackers.append([tracker(),0,None,None,0])
for i in range(len(bbox)):
    ok = mashin_trackers[i][0].init(frame, bbox[i])
    for k,j in enumerate(start):
        if shugam_shalgah(bbox[i], j):
            mashin_trackers[i][2]=k
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Garalt'+tracker_type+'.avi', fourcc, 20.0, (int(width),int(height)))
toologdson = []
frame_det = 1
pbar = tqdm(total=34800)
cnt=0
while True:
    ok, frame = video.read()
    pbar.update(1)
    if cnt%2 == 1:
        cnt+=1
        continue
    if cnt == 1000:
        break
    if not ok:
        out.release()
        break
    if frame_det == 30:
        cv2.imwrite("temp.png", frame)
        nbbox = mashin_detection(frame,bbox)
        for ntemp in nbbox:
            bbox.append(ntemp)
            mashin_trackers.append([tracker(),0,None,None,0])
            mashin_trackers[-1][0].init(frame, bbox[-1])
        frame_det=0
    for i in range(len(bbox)):
        if mashin_trackers[i] is None:
            continue
        ok, bbox[i] = mashin_trackers[i][0].update(frame)
        mashin_trackers[i][4]+=1
        if mashin_trackers[i][4] == 1500:
            mashin_trackers[i]=None
            bbox[i]=None
            continue
        if mashin_trackers[i][2] is None:
            for k,j in enumerate(start):
                if shugam_shalgah(bbox[i], j):
                    mashin_trackers[i][2]=k
                    break
        for k,j in enumerate(end):
            if shugam_shalgah(bbox[i], j):
                mashin_trackers[i][3]=k
                break
        if ok:
            p1 = (int(bbox[i][0]), int(bbox[i][1]))
            p2 = (int(bbox[i][0] + bbox[i][2]), int(bbox[i][1] + bbox[i][3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 4, 1)
        else:
            mashin_trackers[i][1]+=1
        if not mashin_trackers[i][3] is None:
            bbox[i]=None
            toologdson.append([mashin_trackers[i][2],mashin_trackers[i][3]])
            mashin_trackers[i] = None
            continue
        elif mashin_trackers[i][1]>=20:
            mashin_trackers[i] = None
            bbox[i]=None
            continue
    for i in end:
        start_point = i[0]
        end_point = i[1]
        color = (0, 0, 255)  
        thickness = 6
        frame = cv2.line(frame, start_point, end_point, color, thickness)
    for i in start:
        start_point = i[0]
        end_point = i[1]
        color = (0, 255, 0)  
        thickness = 6
        frame = cv2.line(frame, start_point, end_point, color, thickness)
    out.write(frame)
    frame_det+=1
    cnt+=1
    gc.collect()
end_time = time.time()
pbar.close()
print(end_time-start_time)

  0%|          | 4/34800 [00:00<16:36, 34.91it/s]

Unable to show image: No module named 'skimage'


  0%|          | 62/34800 [00:03<1:31:54,  6.30it/s]

Unable to show image: No module named 'skimage'


  0%|          | 122/34800 [00:07<1:16:20,  7.57it/s]

...

Unable to show image: No module named 'skimage'


  3%|▎         | 961/34800 [01:04<2:45:55,  3.40it/s]

Unable to show image: No module named 'skimage'


  3%|▎         | 1001/34800 [01:06<37:28, 15.03it/s] 

70.78879809379578


In [28]:
zug = {"0":"Hoid","1":"Zuun","2":"Urd","3":"Baruun"}
tooloh = {}
for i in range(4):
    tooloh[zug[str(i)]]={}
for i in toologdson:
    if i[0] is None:
        continue
    start_zug = zug[str(i[0])]
    end_zug = zug[str(i[1])]
    
    if tooloh.get(start_zug).get(end_zug):
        tooloh[start_zug][end_zug]+=1
    else:
        tooloh[start_zug][end_zug]=1
for key in tooloh:
    for i in tooloh[key]:
        print(str(key)+"--->"+str(i)+" : "+str(tooloh[key][i]))

Hoid--->Baruun : 5
Hoid--->Urd : 4
Hoid--->Hoid : 1
Zuun--->Zuun : 2
Urd--->Urd : 2
Urd--->Zuun : 4
Baruun--->Urd : 1


In [16]:
toologdson

[[None, 0],
 [None, 0],
 [1, 2],
 [None, 2],
 [3, 0],
 [3, 0],
 [None, 2],
 [None, 0],
 [None, 0],
 [None, 0],
 [3, 0],
 [3, 0],
 [3, 0],
 [None, 0],
 [None, 0],
 [2, 0],
 [2, 0],
 [2, 0],
 [2, 0],
 [None, 0],
 [None, 3],
 [2, 0],
 [2, 0],
 [None, 0],
 [2, 0],
 [None, 0],
 [2, 0],
 [1, 0],
 [2, 0],
 [None, 0],
 [2, 0],
 [2, 0],
 [2, 0],
 [1, 0],
 [2, 0],
 [2, 3],
 [2, 0],
 [2, 0],
 [2, 0],
 [None, 0],
 [3, 3],
 [2, 0],
 [2, 0],
 [2, 0],
 [None, 0],
 [2, 0],
 [2, 0],
 [2, 0],
 [2, 0],
 [2, 0],
 [None, 0],
 [2, 0],
 [1, 0],
 [2, 0],
 [2, 0],
 [None, 0]]

In [14]:
res = [[5,1,None],6,1,None,2,None,None,3]
res = list(filter(None, res))

In [12]:
res

[[5, 1, None], 6, 1, 2, 3]